
# Trying Different ML Techniques for Classification.

### Techniques i'll be using:
* Naive Bayes
* SVM
* LSTM
* Attention model

> For Embedding and Preprocessing, i've referenced the amazing kernels listed here , Do Check Them Out: 
* <a href='https://www.kaggle.com/sudalairajkumar/a-look-at-different-embeddings'>A Look At Different Embeddings</a>
* <a href='https://www.kaggle.com/theoviel/improve-your-score-with-some-text-preprocessing'>Improve Your Score With Some Preprocessing</a>

### First Let's Import The Libraries Needed.


### A Note To Remember : Sincere = 0, Insincere = 1

In [ ]:
# the obvious
import numpy as np
import pandas as pd

# core utility modules
from os import listdir, path
import string
from collections import Counter

# for visualization
import matplotlib.pyplot as plt
from wordcloud import WordCloud
%matplotlib inline

# for preprocessing and feature extraction
import keras.preprocessing.text as text
import keras.preprocessing.sequence as seq 
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# for logging and early stopping and learning rate scheduling
from keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau

# for the metric (F1 Score)
from sklearn.metrics import f1_score

# for model creation and training
from keras.models import Sequential, Model
from keras.layers import Dense, Input, LSTM, Dropout, Bidirectional, CuDNNLSTM
from sklearn.svm import SVC

# other imports
import operator 
import re

### Let's Declare Some Globals

In [ ]:
max_seq_len = 70 # The Max Length Of The Text Sequence
embed_size = 300 # The Number Of Features In The Embedding For A Single Word
max_features = 50000 # The Maximum Number Of Words In The Vocab
EMBEDDING = 'glove.840B.300d' # Learned Embedding To Be Used, Change This For Using Different Embeddings
MODEL = 'attention' # The Model To Use To Classify The Insincere/Sincere Questions, Other Possible Vals Are : 'nb', 'svm', 'lstm'
embedding_matrix = 'None' # The Embeddigns Matrix
embeddings_idx = 'None' # The Mappings From Embedding Index To The Embedding

### Now Let's Get The Data

In [ ]:
listdir('../input')

In [ ]:
train_set = pd.read_csv('../input/train.csv')
test_set = pd.read_csv('../input/test.csv')
train_set.head()

In [ ]:
test_set.head()

### Let's see the distribution

In [ ]:
x = train_set['target'].value_counts(dropna=False)
print(x)
sincere_examples = x[0]
insincere_examples = x[1]

So there are no missing values in the dataset, but, there is a large difference in data distribution as #examples[i==0] >> #examples[i==1]

### Visualizing The Data Separation

In [ ]:
plt.hist(train_set['target'], bins=range(0,6), align='left', rwidth=1)

### Let's Do some EDA
Firstly, Let's See The Word Length Distributions.

In [ ]:
# max and min question lengths
# to remove punctuations : translate(str.maketrans('','',string.punctuation))
lengths_without_puncs = [len(i.translate(str.maketrans('','',string.punctuation)).split()) for i in train_set['question_text']]
lengths = [len(i.split()) for i in train_set['question_text']]
print('With Punctuations: ')
print('Max Length Of Questions: {}'.format(np.max(lengths)))
print('Min Length Of Questions: {}'.format(np.min(lengths)))
print('Without Punctuations: ')
print('Max Length Of Questions: {}'.format(np.max(lengths_without_puncs)))
print('Min Length Of Questions: {}'.format(np.min(lengths_without_puncs)))
# print(len(lengths))

So We Can See That the length of questions range from 0 to 132. We've to remove the empty length questions as they'll not contribute anything to learning. First Let's see how many 0 length questions are there.

In [ ]:
print(len(lengths_without_puncs) - np.count_nonzero(lengths_without_puncs)) # Will remove them or use fillna to overcome this

In [ ]:
plt.hist(lengths)
plt.yscale('log')

So Most Questions Range In Length From 0 to 60.

### Let's Do Some Preprocessing on the data

In [ ]:
# Code from https://www.kaggle.com/theoviel/improve-your-score-with-some-text-preprocessing

contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }

punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

def clean_contractions(text, mapping=contraction_mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

def clean_special_chars(text, punct=punct, mapping=punct_mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}  # Other special characters that I have to deal with in last
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text

def correct_spelling(x, dictionary=mispell_dict):
    for word in dictionary.keys():
        x = x.replace(word, dictionary[word])
    return x


def clean(text):
    text = text.lower()
    text = clean_contractions(text)
    text = clean_special_chars(text)
    text = correct_spelling(text)
    return text

In [ ]:
sincere_counts = Counter()
insincere_counts = Counter()
word_dict = Counter()
sincere_to_insincere_ratio = Counter()

def prepare_dicts():
    qs = [clean(i) for i in train_set['question_text']]
    lbl = [j for j in train_set['target']]
    for i,j in zip(qs,lbl):
        words = i.split()
        # making the dictionaries
        for word in words:
            word_dict[word] += 1
            if j == 0:
                sincere_counts[word] += 1
            elif j == 1:
                insincere_counts[word] += 1
    
    tst_qs = [clean(i) for i in test_set['question_text']]
    
    for i in tst_qs:
        i = i.split()
        for j in i:
            word_dict[j] += 1
    
    print('Words in sincere Questions: {}'.format(len(sincere_counts)))
    print('Words in insincere Questions: {}'.format(len(insincere_counts)))
    print('Total Words in corpus: {}'.format(len(word_dict)))

    print('Most Common Words in Sincere Questions : ')
    print(sincere_counts.most_common()[:10])
    print('Most Common Words in Insincere Questions : ')
    print(insincere_counts.most_common()[:10])

    for i in sincere_counts:
        if sincere_counts[i] >= 100:
            sincere_to_insincere_ratio[i] = np.log(sincere_counts[i]/(insincere_counts[i] + 1))

    print('The Most Sincere Words : ')
    print(sincere_to_insincere_ratio.most_common()[:10])
    print('The Most Insincere Words : ')
    print(list(reversed(sincere_to_insincere_ratio.most_common()))[:10])
    
    return sincere_counts, insincere_counts, word_dict

In [ ]:
prepare_dicts()

### Aaannddd a wordcloud for fun

In [ ]:
wordCloud = WordCloud().generate(" ".join([key[0] for key in sincere_to_insincere_ratio.most_common()[:10]]))
fig = plt.figure()
plt.imshow(wordCloud, interpolation="bilinear")
plt.axis("off")
plt.margins(x=0, y=0)
fig.suptitle('Most Common Words In Sincere Questions', fontsize=14, fontweight='bold')
plt.show()

In [ ]:
wordCloud = WordCloud().generate(" ".join([key[0] for key in list(reversed(sincere_to_insincere_ratio.most_common()))[:10]]))
fig = plt.figure()
plt.imshow(wordCloud, interpolation="bilinear")
plt.axis("off")
plt.margins(x=0, y=0)
fig.suptitle('Most Common Words In Insincere Questions', fontsize=14, fontweight='bold')
plt.show()

In [ ]:
# Creating The Datasets First
train_x = list(train_set['question_text'].fillna("_na_").values)
train_y = list(train_set['target'])

test_x = list(test_set['question_text'].fillna("_na_").values)

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2)

# Cleaning Up The Data (train + test)
train_x = [clean(i) for i in train_x]
val_x = [clean(i) for i in val_x]
test_x = [clean(i) for i in test_x]

# An Example From Train Set
print('An Example From Train Set: ')
print(train_x[0])

## Tokenize the sentences
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_x))
train_X = tokenizer.texts_to_sequences(train_x)
val_X = tokenizer.texts_to_sequences(val_x)
test_X = tokenizer.texts_to_sequences(test_x)

# After Tokenizing
print('After Tokenizing: ')
print(train_X[0])

## Pad the sentences 
train_X = seq.pad_sequences(train_X, maxlen=max_seq_len)
val_X = seq.pad_sequences(val_X, maxlen=max_seq_len)
test_X = seq.pad_sequences(test_X, maxlen=max_seq_len)

# After Padding
print('After Padding: ')
print(train_X[0])

print(np.shape(train_X), np.shape(train_y), np.shape(val_X), np.shape(val_y))

### Let's Get Those Embeddings

In [ ]:
# Thanks to https://www.kaggle.com/sudalairajkumar/a-look-at-different-embeddings

def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')

def get_embeddings(embedding_name, mode='new'):
    # Getting The File
    filePath = '../input/embeddings/{0}/{0}.txt'.format(embedding_name)
    
    # Creating a Dictionary of format {word : Embedding}
    if mode == 'new':
        embeddings_idx = dict(get_coefs(*i.split(" ")) for i in open(filePath))
        # All Embeddings
        all_embs = np.stack(embeddings_idx.values())

        # Creating The Embedding Matrix with distribution, for if there is a missing word in the embeddings, it'll have
        # the embedding vector with the same distribution
        emb_mean,emb_std = all_embs.mean(), all_embs.std()
        embed_size = all_embs.shape[1]

        word_index = tokenizer.word_index
        nb_words = min(max_features, len(word_index))
        embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))

        # Filling in the given learned embeddings in the embedding matrix
        for word, i in word_index.items():
            if i >= max_features: continue
            embedding_vector = embeddings_idx.get(word)
            if embedding_vector is not None: embedding_matrix[i] = embedding_vector
                
    return embeddings_idx, embedding_matrix
# # Creating The Embedding Matrix From The Given Embedding
# get_embeddings(EMBEDDING)

In [ ]:
# Checking OOV words (Out Of Vocab words)
def check_coverage(vocab, embeddings_index):
    known_words = {}
    unknown_words = {}
    nb_known_words = 0
    nb_unknown_words = 0
    for word in vocab.keys():
        try:
            known_words[word] = embeddings_index[word]
            nb_known_words += vocab[word]
        except:
            unknown_words[word] = vocab[word]
            nb_unknown_words += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(known_words) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(nb_known_words / (nb_known_words + nb_unknown_words)))
    unknown_words = sorted(unknown_words.items(), key=operator.itemgetter(1))[::-1]

    return unknown_words

In [ ]:
embedding_idxs, embedding_mtx = get_embeddings(EMBEDDING, 'new')
unk_wrds = check_coverage(word_dict, embedding_idxs)

In [ ]:
print(unk_wrds[:10])

### Enough Chit-Chat, Let's Start Making Some Models And Getting Some Results

Creating a common class structure for all models for ease of use

In [ ]:
# Defining The NaiveBayes Class
class NaiveBayes():
    def __init__(self):
        self.sincere_example_count = sincere_examples
        self.insincere_example_count = insincere_examples
        self.total_examples = x[0]+x[1]
        self.sincere_dict = sincere_counts
        self.insincere_dict = insincere_counts
        self.word_dict= word_dict
        self.sincere_word_count = np.sum(list(sincere_counts.values()))
        self.insincere_word_count = np.sum(list(insincere_counts.values()))
        self.sincere_prob = self.sincere_example_count / self.total_examples
        self.insincere_prob = self.insincere_example_count / self.total_examples
    
    def summary(self):
        print('Positive Examples : {}, Negative Examples : {}, Total Examples : {}'.format(self.sincere_example_count, self.insincere_example_count, self.total_examples))
    
    def compile(self):
        # Compilation step
        # Do Nothing
        print('Compiling...')
    
    def fit(self, x_train, y_train, batch_size):
        # NB Model Fitting
        print(np.shape(x_train), np.shape(y_train))
        num_batches = int(len(x_train)/(batch_size*10))
        for epoch in range(num_batches):
            indices = np.random.randint(0,np.shape(y_train)[0], batch_size)
            examples = [x_train[i] for i in indices]
            labels = [y_train[j] for j in indices]
            predictions = []
            
            for i in examples:
                #                   P(words|c=1)*P(c=1) + 1
                # P(c=1|words) =   ------------------------
                #                       P(words) + 2
                #
                #                   P(words|c=0)*P(c=0) + 1
                # P(c=0|words) =   ------------------------
                #                       P(words) + 2
                #
                # Where P(words|c=i) = product(P(word[j]|c=i))
                # 
                # And P(words) = product(P(word[j]))
                #
                # 1 and 2 are added to numerator and denominator acc to laplace smoothing, 
                # to avoid division by zero or numerator being 0
                p_words = np.prod([word_dict[j]/np.sum(list(word_dict.values())) for j in i.split()])
                p_words += 2
                sincere_prob_num = np.prod([sincere_counts[j]/self.sincere_word_count for j in i.split()]) * self.sincere_prob
                insincere_prob_num = np.prod([insincere_counts[j]/self.insincere_word_count for j in i.split()]) * self.insincere_prob
                
                sincere_prob = sincere_prob_num/p_words
                insincere_prob = insincere_prob_num/p_words
                
                print('Sincere_prob: {}, Insincere_prob: {}'.format(sincere_prob, insincere_prob))
                predictions.append(np.argmax([sincere_prob, insincere_prob]))
            
            f1 = f1_score(labels, predictions)
            print('epoch {}/{}, f1_score : {}'.format(epoch, num_batches, f1))
            
    def evaluate(self, x_val, y_val):
        predictions = []
        for example in x_val:
            p_words = np.prod([word_dict[j]/np.sum(list(word_dict.values())) for j in example.split()])
            p_words += 2
            sincere_prob_num = np.prod([sincere_counts[j]/self.sincere_word_count for j in example.split()]) * self.sincere_prob + 1
            insincere_prob_num = np.prod([insincere_counts[j]/self.insincere_word_count for j in example.split()]) * self.insincere_prob + 1
            
            sincere_prob = sincere_prob_num/p_words
            insincere_prob = insincere_prob_num/p_words
            
            predictions.append(np.argmax([pos_prob, neg_prob]))
        
        f1 = f1_score(y_val, predictions)
        print(f1)
        
    
    def predict(self, x_test):
        # The NB Prediction with Laplace Smoothing
        predictions = []
        for example in x_test:
            p_words = np.prod([word_dict[j]/np.sum(list(word_dict.values())) for j in example.split()])
            p_words += 2
            sincere_prob_num = np.prod([sincere_counts[j]/self.sincere_word_count for j in example.split()]) * self.sincere_prob
            insincere_prob_num = np.prod([insincere_counts[j]/self.insincere_word_count for j in example.split()]) * self.insincere_prob

            sincere_prob = sincere_prob_num/p_words
            insincere_prob = insincere_prob_num/p_words

#             print('Sincere_prob: {}, Insincere_prob: {}'.format(sincere_prob, insincere_prob))
            predictions.append(np.argmax([sincere_prob, insincere_prob]))
#             print('predicted Class : {}'.format(np.argmax([sincere_prob, insincere_prob])))
        return predictions

# The SVM Class
class __SVC__(SVC):
    def __init__(self):
        super(__SVC__,self).__init__(verbose=True)
        print('initializing...')
    
    def summary(self):
        print(self.__dict__)
        
    def prepare_data(self, X_train, X_val, X_test):
        self.X_train = [embedding_idxs[word] for example in X_train for word in example.split()]
        self.X_val = [embedding_idxs[word] for example in X_val for word in example.split()]
        self.X_test = [embedding_idxs[word] for example in X_test for word in example.split()]
        
        # Pad The Data
#         self.X_train = 

In [ ]:
# The Model Class
class __Model__():
    def __init__(self, model=MODEL):
        self.model_type = model
        self.model = self.get_model()
    
    # Create The Model
    def get_model(self):
        if self.model_type == 'nb':
            # create naivebayes model
            model = NaiveBayes()
            
        elif self.model_type == 'svm':
            # create svm model
            model = __SVC__()
            
        elif self.model_type == 'lstm':
            # create lstm model
            model = Sequential()
            model.add(LSTM())
            model.add(LSTM())
            model.add(Dense())
            
        else:
            # create attention model
            model = Sequential()
            model.add(LSTM())
            model.add(Bidirectional())
            model.add(Attention())
            model.add(Dense())
            
        return model
                
    # Compile The Model
    def compile(self, optimizer='adam', loss='categorical_crossentropy'):
        # Model Compilation
        print('Compiling...')
    
    def fit(self, x_train, y_train, batch_size=128):
        # Model Training
        print('Fitting...')
        self.model.fit(x_train, y_train, batch_size)
    
    def predict(self, x_test):
        # Predictions from the model
        print('Predicting...')
        return self.model.predict(x_test)
        
    def evaluate(self, x_eval, y_eval):
        # Model evaluation
        print('Evaluating...')
        self.model.evaluate(x_eval, y_eval)
    
    def save_model(self):
        # Save Model Weights
        print('Saving...')
        
    def summary(self):
        return self.model.summary()
    


## Naive Bayes, Working Without Embeddings

In [ ]:
nb = __Model__('nb')
nb.summary()
# nb.fit(train_x, train_y)
# nb.prepare_data()
# nb.fit(train_x,train_y)
# nb.evaluate(val_x, val_y)

## SVM, Simple ML Classifier

In [ ]:
svm = __Model__('svm')
svm.summary()
svm.prepare_data()
svm.fit(svm.x_train, svm.y_train)
svm.evaluate(svm.x_val, svm.y_val)

## LSTM it is.

In [ ]:
lstm = __Model__('lstm')
lstm.summary()
lstm.prepare_data()
lstm.compile()
lstm.fit(train_X, train_Y)

## Attention Folks!

In [ ]:
attention = __Model__('attention')
attention.summary()
attention.prepare_data()
attention.compile()
attention.fit()

## Comparing All Performances

Score Obtained From Naive Bayes : 
Score Obtained From SVM : 
Score Obtained From Simple LSTM :
Score Obtained From Attention Model : 

## That's it. Let's Create The Submission Files From Each Model.
🤞

In [ ]:
# 1. Naive Bayes
predictions_nb = nb.predict(test_x)

predictions_nb = pd.Series(predictions_nb,name="prediction")
predictions_nb = pd.concat([pd.Series(range(1,test_set['qid'].count()),name = "qid"),predictions_nb],axis = 1)

print(predictions_nb)

predictions_nb.to_csv('nb_pred.csv',index=False)
    

# 2. SVM
# predictions_svm = svm.predict(test_X)

# 3. LSTM
# predictions_lstm = lstm.predict(test_X)

# 4. Attention
# predictions_attention = attention.predict(test_X)


# Saving all prediction in the submissions file format
